In [1]:
#!pip install ipykernel ipython --upgrade

asyncio doesn't work with tornado > 5.0 yet...
See, e.g. https://github.com/jupyter/notebook/issues/3397

In [2]:
#!pip install tornado==4.5.3

asyncio “provides infrastructure for writing single-threaded concurrent code using coroutines.
The asyncio module provides a framework that revolves around the event loop. An event loop basically says “when event A happens, react with function B”.

When a user loads the web page, the server will check for and call one or more event handlers. Once those event handlers are done, they need to give control back to the event loop. To do this in Python, asyncio uses coroutines.

A coroutine is a special function that can give up control to its caller without losing its state.

In [1]:
%autoawait False

In [2]:
%autoawait

IPython autoawait is `off`, and set to use `asyncio`


In [3]:
import asyncio

Examples are here:
https://www.blog.pythonlibrary.org/2016/07/26/python-3-an-intro-to-asyncio/

To define a coroutine with asyncio, you can use the async def statement:

In [6]:
async def hello():
        print("Hello, async!")

coro = hello()
coro
# Output

<coroutine object hello at 0x7f86411d01a8>

The asyncio coroutines do not support
next, but they can be easily run in the asyncio event loop using the
run_until_complete method:

In [7]:
loop = asyncio.get_event_loop()
loop.run_until_complete(coro)

Hello, async!


The asyncio  module provides resources (called awaitables) that can be requested inside
coroutines through the await syntax. For example, if we want to wait for a certain time and
then execute a statement, we can use the asyncio.sleep function:

In [8]:
async def wait_and_print(msg):
        await asyncio.sleep(1)
        print("Message: ", msg)

In [9]:
loop = asyncio.get_event_loop()

In [11]:
loop.run_until_complete(wait_and_print("Hello"))

Message:  Hello


The result is beautiful, clean code. We are writing perfectly functional asynchronous code
without all the ugliness of callbacks!

You may have noted how await provides a breakpoint for the event loop
so that, as it wait for the resource, the event loop can move on and
concurrently manage other coroutines

Even better, coroutines are also awaitable, and we can use the await statement to chain
coroutines asynchronously. In the following example, we rewrite the
network_request function, which we defined earlier, by replacing the call to time.sleep
with asyncio.sleep:

In [13]:
async def network_request(number):
         await asyncio.sleep(1.0)
         return {"success": True, "result": number ** 2}

We can follow up by reimplementing fetch_square. As you can see, we can await
network_request directly without needing additional futures or callbacks.

In [14]:
async def fetch_square(number):
         response = await network_request(number)
         if response["success"]:
             print("Result is: {}".format(response["result"]))

The coroutines can be executed individually using loop.run_until_complete:

In [17]:
loop.run_until_complete(fetch_square(2))
loop.run_until_complete(fetch_square(3))
loop.run_until_complete(fetch_square(4))

Result is: 4
Result is: 9
Result is: 16


Running tasks using run_until_complete is fine for testing and debugging. However,
our program will be started with loop.run_forever most of the times, and we will need
to submit our tasks while the loop is already running.
asyncio provides the ensure_future function, which schedules coroutines (as well as
futures) for execution. ensure_future can be used by simply passing the coroutine we
want to schedule. The following code will schedule multiple calls to fetch_square that
will be executed concurrently:

In [18]:
asyncio.ensure_future(fetch_square(2))
asyncio.ensure_future(fetch_square(3))
asyncio.ensure_future(fetch_square(4))

<Task pending coro=<fetch_square() running at <ipython-input-14-b131e5135712>:1>>

In [19]:
loop.run_forever()

Result is: 4
Result is: 9
Result is: 16


KeyboardInterrupt: 

In [4]:
import aiohttp
import asyncio
import async_timeout
import os
 
 
async def download_coroutine(session, url):
    with async_timeout.timeout(10):
        async with session.get(url) as response:
            filename = os.path.basename(url)
            with open(filename, 'wb') as f_handle:
                while True:
                    chunk = await response.content.read(1024)
                    if not chunk:
                        break
                    f_handle.write(chunk)
            return await response.release()
 
 
async def main(loop):
    urls = ["http://www.irs.gov/pub/irs-pdf/f1040.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040a.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040ez.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040es.pdf",
        "http://www.irs.gov/pub/irs-pdf/f1040sb.pdf"]
 
    async with aiohttp.ClientSession(loop=loop) as session:
        tasks = [download_coroutine(session, url) for url in urls]
        await asyncio.gather(*tasks)
 
 


In [5]:
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

You can also schedule calls to regular functions using the asyncio event loop. The first method we’ll look at is call_soon. The call_soon method will basically call your callback or event handler as soon as it can. It works as a FIFO queue, so if some of the callbacks take a while to run, then the others will be delayed until the previous ones have finished. Let’s look at an example:

In [6]:
import asyncio
import functools
 
 
def event_handler(loop, stop=False):
    print('Event handler called')
    if stop:
        print('stopping the loop')
        loop.stop()
 
 
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    try:
        loop.call_soon(functools.partial(event_handler, loop))
        print('starting event loop')
        loop.call_soon(functools.partial(event_handler, loop, stop=True))
 
        loop.run_forever()
    finally:
        print('closing event loop')
        loop.close()

starting event loop
Event handler called
Event handler called
stopping the loop
closing event loop


There is a related function called call_soon_threadsafe. As the name implies, it works the same way as call_soon, but it’s thread-safe.

In [26]:
import asyncio
 
 
async def my_task(seconds):
    """
    A task to do for a number of seconds
    """
    print('This task is taking {} seconds to complete'.format(
        seconds))
    await asyncio.sleep(seconds)
    return 'task finished'
 
 
if __name__ == '__main__':
    my_event_loop = asyncio.get_event_loop()
    try:
        print('task creation started')
        task_obj = my_event_loop.create_task(my_task(seconds=2))
        my_event_loop.run_until_complete(task_obj)
    finally:
        my_event_loop.close()
 
    print("The task's result was: {}".format(task_obj.result()))

task creation started
This task is taking 2 seconds to complete
The task's result was: task finished
